In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import time
import datetime
import pickle
import ta 
import gc
from tqdm import tqdm

TRAIN_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\train.csv'
TRAIN_COMBINE_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\train_combine.csv'
TEST_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\test.csv'
ADDITIONAL_TRAIN_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\train2.csv'
MODELS_DIR = r'C:\Users\e0817820\Desktop\tokka\models'

C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\1875196596.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
use_additional_train = False

In [3]:
list_cryptocurrencies = ['AVAX', 'ADA', 'SOL', 'BNB', 'TRX', 'DOGE', 'LINK', 'XRP', 'BTC', 'ETH']
df_train = pd.read_csv(ADDITIONAL_TRAIN_CSV)
df_test = pd.read_csv(TEST_CSV)
lower_limit_test = df_test['timestamp'].min()
upper_limit_train = df_train['timestamp'].max()
lower_limit_test_datetime = pd.to_datetime(df_test['timestamp'].min(), unit='ms')
upper_limit_train_datetime = pd.to_datetime(df_train['timestamp'].max(), unit='ms')
# df_train.drop(columns=['Unnamed: 0'])

In [4]:
print(df_test.shape)

(1226990, 12)


In [5]:
df_combined = pd.concat([df_train, df_test])
df_combined['timestamp'] = pd.to_datetime(df_combined['timestamp'], unit='ms')
all_timestamps = pd.date_range(start=upper_limit_train_datetime, end=lower_limit_test_datetime, freq='T')  # Adjust the freq='T' as necessary

expanded_df = pd.DataFrame()
for symbol in list_cryptocurrencies:
    temp_df = pd.DataFrame({'timestamp': all_timestamps})
    temp_df['symbol'] = symbol 
    temp_df = pd.merge(temp_df, df_combined[df_combined['symbol'] == symbol], on='timestamp', how='left', suffixes=('', '_drop'))
    temp_df.drop([col for col in temp_df if col.endswith('_drop')], axis=1, inplace=True)
    expanded_df = pd.concat([expanded_df, temp_df])

expanded_df = expanded_df.sort_values(by=['timestamp', 'symbol'], ascending=[True, True])
expanded_df['timestamp'] = (expanded_df['timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

df_filled = pd.concat([expanded_df, df_test])
df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))
df_filled.reset_index(drop=True, inplace=True)
df_filled.head()
del df_combined

df_filled.rename(columns={'index': 'timestamp'}, inplace=True)
df_filled.drop(columns=['id'], inplace=True)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\3585968195.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  all_timestamps = pd.date_range(start=upper_limit_train_datetime, end=lower_limit_test_datetime, freq='T')  # Adjust the freq='T' as necessary
C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\3585968195.py:17: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))
C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\3585968195.py:17: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))
C:\Users

In [6]:
print(df_filled.shape)

(1233320, 12)


In [7]:
df_filled.to_csv(r'C:\Users\e0817820\Desktop\tokka\data\raw\test_filled.csv', index=False)

In [8]:
df_filled.head()

,timestamp,symbol,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_volume,taker_sell_volume,log_return
0,1691345340000,ADA,0.292300,0.292400,0.292300,0.292400,3343.500000,977.553110,34.000000,2151.300000,629.040120,0.0
1,1691345400000,ADA,0.292303,0.292403,0.292303,0.292403,3353.617405,980.541681,33.963608,2156.203006,630.490373,0.0
2,1691345460000,ADA,0.292306,0.292406,0.292306,0.292406,3363.734810,983.530252,33.927215,2161.106013,631.940626,0.0
3,1691345520000,ADA,0.292309,0.292409,0.292309,0.292409,3373.852215,986.518824,33.890823,2166.009019,633.390879,0.0
4,1691345580000,ADA,0.292313,0.292413,0.292313,0.292412,3383.969620,989.507395,33.854430,2170.912025,634.841132,0.0


In [9]:
combined_df = pd.DataFrame()
combined_df[df_filled.columns] = 0
for id in tqdm(range(10)):
    combined_df = combined_df.merge(df_filled.loc[df_filled["symbol"] == list_cryptocurrencies[id], ['timestamp', 'high', 'low', 'close', 'log_return']].copy(), on="timestamp", how='outer', suffixes=['', "_"+list_cryptocurrencies[id]])
        
combined_df = combined_df.drop(df_filled.columns.drop("timestamp"), axis=1)
combined_df = combined_df.sort_values('timestamp', ascending=True)
combined_df = combined_df.fillna(method='ffill')
combined_df.drop_duplicates(subset=['timestamp'], inplace=True)

combined_df = combined_df[combined_df['timestamp'] > lower_limit_test - 400*60*1000]
display(combined_df.head())

100%|██████████| 10/10 [00:01<00:00,  7.46it/s]
C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\2246882315.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df = combined_df.fillna(method='ffill')


,timestamp,high_AVAX,low_AVAX,close_AVAX,log_return_AVAX,high_ADA,low_ADA,close_ADA,log_return_ADA,high_SOL,...,close_XRP,log_return_XRP,high_BTC,low_BTC,close_BTC,log_return_BTC,high_ETH,low_ETH,close_ETH,log_return_ETH
233,1691359320000,12.637927,12.637927,12.637927,-0.000795,0.293137,0.293037,0.293100,0.0,23.237927,...,0.627510,-0.000478,29131.841044,29131.831044,29131.841044,-0.000027,1836.244019,1836.230332,1836.240332,0.000464
234,1691359380000,12.638133,12.638133,12.638133,-0.000795,0.293141,0.293041,0.293103,0.0,23.238133,...,0.627508,-0.000478,29131.977658,29131.967658,29131.977658,-0.000027,1836.256139,1836.242437,1836.252437,0.000464
235,1691359440000,12.638339,12.638339,12.638339,-0.000795,0.293144,0.293044,0.293106,0.0,23.238339,...,0.627505,-0.000478,29132.114272,29132.104272,29132.114272,-0.000027,1836.268259,1836.254541,1836.264541,0.000464
236,1691359500000,12.638544,12.638544,12.638544,-0.000795,0.293147,0.293047,0.293109,0.0,23.238544,...,0.627503,-0.000478,29132.250886,29132.240886,29132.250886,-0.000027,1836.280380,1836.266646,1836.276646,0.000464
237,1691359560000,12.638750,12.638750,12.638750,-0.000795,0.293150,0.293050,0.293112,0.0,23.238750,...,0.627500,-0.000478,29132.387500,29132.377500,29132.387500,-0.000027,1836.292500,1836.278750,1836.288750,0.000464


In [10]:
print(combined_df.shape)

(123098, 41)


In [12]:
def bollinger_mavg(close, window=20):
    """Calculate the middle Bollinger Band."""
    return close.rolling(window=window).mean()

def bollinger_hband(close, window=20, k=2):
    """Calculate the upper Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg + (mstd * k)

def bollinger_lband(close, window=20, k=2):
    """Calculate the lower Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg - (mstd * k)

def bollinger_wband(close, window=20, k=2):
    """Calculate the width of the Bollinger Bands."""
    hband = bollinger_hband(close, window, k)
    lband = bollinger_lband(close, window, k)
    mavg = bollinger_mavg(close, window)
    return ((hband - lband) / mavg) * 100

def exponential_moving_average(close, span=20):
    """Calculate the Exponential Moving Average (EMA)."""
    return close.ewm(span=span, adjust=False).mean()

def relative_strength_index(close, window=14):
    """Calculate the Relative Strength Index (RSI)."""
    delta = close.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi
  
def stochastic_oscillator(high, low, close, window=14):
    """Calculate the Stochastic Oscillator."""
    low_min = low.rolling(window=window).min()
    high_max = high.rolling(window=window).max()
    k = ((close - low_min) / (high_max - low_min)) * 100
    return k

def williams_r(high, low, close, window=14):
    """Calculate Williams %R."""
    high_max = high.rolling(window=window).max()
    low_min = low.rolling(window=window).min()
    r = -100 * ((high_max - close) / (high_max - low_min))
    return r
  
def rate_of_change(close, window=14):
    """Calculate the Rate of Change (ROC)."""
    roc = ((close - close.shift(window)) / close.shift(window)) * 100
    return roc

In [13]:
def getFeatures(df, limit):
    lags_normal = [8, 20, 120]
    lags_ema = [15]
    lags_rsi = [8, 14]
    lags_stoch_osci = [10, 25]
    roc_window = [8, 15]
    williams_percentage_r = [10, 20]
    lags_wbands = [10, 50]
    for id in range(10):        
        symbol = list_cryptocurrencies[id] 
        for lag in lags_normal:
            df[f'log_close/mean_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.append(np.convolve( np.array(df[f'close_{symbol}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1)  )
            df[f'log_return_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.array(df[f'close_{symbol}']), lag)  )
    for lag in lags_normal:
        df[f'mean_close/mean_{lag}'] =  np.mean(df.iloc[:,df.columns.str.startswith(f'log_close/mean_{lag}_')], axis=1)
        df[f'mean_log_returns_{lag}'] = np.mean(df.iloc[:,df.columns.str.startswith(f'log_return_{lag}_')], axis=1)
        for id in range(10):
            symbol = list_cryptocurrencies[id] 
            df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
            df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
    # apply TA indicators: EMA, RSI, Bollinger Bands, Stochastic Oscillator
    for id in range(10):
        symbol = list_cryptocurrencies[id]
        for lag in lags_ema:
            # df[f'ema_close_{lag}_{symbol}'] = ta.trend.ema_indicator(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'ema_close_{lag}_{symbol}'] = exponential_moving_average(df[f'close_{symbol}'], span=lag)
        for lag in lags_rsi:
            # df[f'rsi_close_{lag}_{symbol}'] = ta.momentum.rsi(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'rsi_close_{lag}_{symbol}'] = relative_strength_index(df[f'close_{symbol}'], window=lag)
        for lag in lags_stoch_osci:
            # df[f'stoch_oscil_close_{lag}_{symbol}'] = ta.momentum.stoch(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'stoch_oscil_close_{lag}_{symbol}'] = stochastic_oscillator(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag)
        for lag in roc_window:
            # df[f'roc_close_{lag}_{symbol}'] = ta.momentum.roc(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'roc_close_{lag}_{symbol}'] = rate_of_change(df[f'close_{symbol}'], window=lag)
        for lag in williams_percentage_r:
            # df[f'williams_close_{lag}_{symbol}'] = ta.momentum.williams_r(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], lbp=lag, fillna=False)  
            df[f'williams_close_{lag}_{symbol}'] = williams_r(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag)
        for lag in lags_wbands:
            # df[f'wbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_wband(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'wbbands_close_{lag}_{symbol}'] = bollinger_wband(df[f'close_{symbol}'], window=lag)
    gc.collect()  
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    for symbol in list_cryptocurrencies:
        df.drop(columns=[f'high_{symbol}', f'low_{symbol}', f'close_{symbol}'], inplace=True)
    gc.collect()  
    df = df[df['timestamp'] >= limit] 
    df.drop_duplicates(subset=['timestamp'], inplace=True)
    return df   

In [14]:
combined_df = getFeatures(combined_df, lower_limit_test)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\2107353982.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\2107353982.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
C

In [15]:
def customMean(preds):
    mean_preds = []
    num_entries = len(preds[0])    
    for i in range(num_entries):
        countNegative = 0
        countPositive = 0
        neg, pos = [], []
        for j in range(len(preds)):
            if preds[j][i] > 0:
                countPositive += 1
                pos.append(preds[j][i])
            elif preds[j][i] < 0:
                countNegative += 1
                neg.append(preds[j][i])
        if countPositive > countNegative:
            mean_preds.append(np.mean(pos))
        elif countPositive < countNegative:
            mean_preds.append(np.mean(neg))
        else:
            mean_preds.append(np.mean(neg+pos))
    return mean_preds


In [16]:
def predictForEachAsset(df):
    preds = []
    output = pd.DataFrame()
    for id in range(10):
        
        pred_asset = []
        symbol = list_cryptocurrencies[id]
        target = f'log_return_{symbol}'
        columns = [col for col in df.columns if symbol in col] + ['timestamp']
        # if symbol != 'BTC':
        #     columns += [col for col in df.columns if 'BTC' in col]
        df_asset = df[columns]
        # df_asset.to_csv(f'C:\\Users\\e0817820\\Desktop\\tokka\\data\\processed\\test_{symbol}.csv', index=False)
        
        #load models
        models = []
        for fold in range(10):
            model = pickle.load(open(os.path.join(MODELS_DIR, f'{symbol}_fold{fold}2.pkl'), 'rb'))
            models.append(model)
        #predict
        df_asset = df_asset.fillna(method='ffill')
        df_asset = df_asset.drop(columns=[target, 'timestamp']) 
        for model in models: 
            pred_fold = model.predict(df_asset)
            pred_asset.append(pred_fold)
        pred = np.mean(pred_asset, axis=0)
        # pred = customMean(pred_asset)
        del models
        preds.append(pred)
        output_symbol = pd.DataFrame({'timestamp': df['timestamp'], 'symbol': symbol, 'log_return': pred})
        output_symbol.fillna(method='bfill', inplace=True)
        output_symbol.fillna(method='ffill', inplace=True)
        output = pd.concat([output, output_symbol])
        gc.collect()
    #join output with df_test on timestamp, symbol
    output.to_csv(r'C:\Users\e0817820\Desktop\tokka\data\processed\output.csv', index=False)
    output = df_test.merge(output, on=['timestamp', 'symbol'], how='left')
    print(output.shape)
    return output

In [17]:
output = predictForEachAsset(combined_df)
output = output[['id', 'log_return']]


C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\794497410.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fillna(method='ffill')
C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\794497410.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='bfill', inplace=True)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\794497410.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ffill', inplace=True)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_21736\794497410.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fil

(1226990, 13)


In [18]:
del combined_df

In [19]:
print(output.shape)

(1226990, 2)


In [20]:
output.to_csv(r'C:\Users\e0817820\Desktop\tokka\data\processed\submission.csv', index=False)